## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[rep_mobile_citizens_id_city_partner]]

[[sessions_st_mobile]]

[[subscriptions_st_mobile]]


### Table

In [5]:
query_text = """--sql
CREATE TABLE db1.total_active_users_per_day_full_table
(
    `report_date` Date,
    `partner_uuid` String,
    `city` String,
    `citizen_id` Int32,
    `monetization` UInt8,
    `subscriptions_state` String,
    `address_uuid` String,
    `flat_uuid` String,
    `created_at` String,
    `activated_at` DateTime
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [13]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.total_active_users_per_day_full_table_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR 10 MINUTE TO db1.total_active_users_per_day_full_table AS
       SELECT
        r_m_c.partner_uuid AS partner_uuid,
        ses_st_m.citizen_id AS citizen_id,
        r_m_c.report_date AS report_date,
        r_m_c.monetization AS monetization,
        sub_st_m.state AS subscriptions_state,
        r_m_c.city AS city,
        r_m_c.address_uuid AS address_uuid,
        r_m_c.flat_uuid AS flat_uuid,
        c_d_m.created_at AS created_at,
        r_m_c.activated_at AS activated_at
    FROM db1.rep_mobile_citizens_id_city_partner as r_m_c
    JOIN db1.`sessions_st_mobile_ch` AS ses_st_m
        ON r_m_c.citizen_id = ses_st_m.citizen_id
        AND r_m_c.report_date = ses_st_m.report_date 
    LEFT JOIN db1.`subscriptions_st_mobile_ch` AS sub_st_m
        ON ses_st_m.citizen_id = sub_st_m.citizen_id
        AND ses_st_m.report_date = sub_st_m.report_date
    LEFT JOIN db1.`citizens_dir_mobile_ch` AS c_d_m ON c_d_m.`citizen_id`  = r_m_c.`citizen_id`
    WHERE 
        toDate(`last_use`) 
        BETWEEN toStartOfMonth(r_m_c.report_date) and r_m_c.report_date 
        AND  r_m_c.state = 'activated'
    --
     SETTINGS join_algorithm = 'partial_merge'
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [3]:
query_text = """--sql
    SELECT
        *
    FROM db1.total_active_users_per_day_full_table
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,partner_uuid,city,citizen_id,monetization,subscriptions_state,address_uuid,flat_uuid,created_at,activated_at
date,str,str,i64,i64,str,str,str,str,datetime[ns]
2025-08-14,"""00c45794-0060-4b37-88bf-3b8dff…","""Зеленодольск""",2037046,0,"""activated""","""8bef7cd0-4758-44bc-bfd4-eec897…","""d40d1870-9f6a-4525-ae1a-bf9484…","""2025-05-31 21:37:07""",2025-05-31 21:37:21
2025-08-14,"""00c45794-0060-4b37-88bf-3b8dff…","""Зеленодольск""",2038459,0,"""activated""","""8bef7cd0-4758-44bc-bfd4-eec897…","""ee304790-f7fb-4bd3-957e-1bf1ee…","""2025-06-02 11:34:04""",2025-06-02 18:53:07
2025-08-14,"""00c45794-0060-4b37-88bf-3b8dff…","""Зеленодольск""",2039173,0,"""""","""8bef7cd0-4758-44bc-bfd4-eec897…","""ee304790-f7fb-4bd3-957e-1bf1ee…","""2025-06-02 19:41:02""",2025-06-02 19:41:18
2025-08-14,"""00c45794-0060-4b37-88bf-3b8dff…","""Зеленодольск""",1785986,0,"""""","""8bef7cd0-4758-44bc-bfd4-eec897…","""1d27ba27-d18a-45eb-adff-7eebcf…","""2024-11-14 20:06:21""",2024-11-16 12:54:01
2025-08-14,"""00c45794-0060-4b37-88bf-3b8dff…","""Зеленодольск""",1792014,0,"""activated""","""8bef7cd0-4758-44bc-bfd4-eec897…","""52a6e3eb-4d0b-444e-8b28-bdb0fc…","""2024-11-19 13:17:32""",2024-11-23 10:24:09
…,…,…,…,…,…,…,…,…,…
2025-08-14,"""00ec6d8e-a799-409c-958b-9472a1…","""Казань""",1976208,0,"""activated""","""ea2fdde0-9343-4be5-a582-4d376b…","""c313e36f-053c-4a19-927c-e42195…","""2025-04-10 13:07:59""",2025-04-10 16:10:03
2025-08-14,"""00ec6d8e-a799-409c-958b-9472a1…","""Казань""",2047579,0,"""""","""6e442211-1689-46bd-a0fb-3e9cd9…","""354550ec-eed6-4686-9945-a8ee3a…","""2025-06-09 20:39:41""",2025-06-09 20:40:04
2025-08-14,"""00ec6d8e-a799-409c-958b-9472a1…","""Казань""",2045922,0,"""""","""29b27b03-c853-44d4-a57a-a1df3f…","""ec3bbb99-2f66-4e15-b2be-62ae7d…","""2025-06-08 10:58:51""",2025-06-08 10:59:23


### delete a part


In [22]:
query_text = """--sql
    ALTER TABLE db1.total_active_users_per_day_full_table DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [12]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_full_table_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [4]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_full_table
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.total_active_users_per_day_full_table_mv
"""

ch.query_run(query_text)